# 9. PMML 模型格式转换

In [1]:
# !pip install nyoka

In [2]:
DIRECTORY = './data'
JOBLIB_FILE = 'model_select.joblib'
TEST_FILE = 'adult/adult.test'
KKV_FILE = 'label_encoder_deploy.json'
PMML_FILE = 'model_select.pmml'
COLS = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
        'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
        'hours-per-week', 'native-country', 'income']
LABEL_COL = 'income'

In [3]:
import os

if os.environ.get('MKL_THREADING_LAYER') is None:
    os.environ['MKL_THREADING_LAYER'] = 'GNU'
print(f"MKL_THREADING_LAYER = {os.environ['MKL_THREADING_LAYER']}")

MKL_THREADING_LAYER = GNU


In [4]:
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from nyoka import lgb_to_pmml
import joblib

import util

## 一、导入模型

In [5]:
joblib_path = util.gen_abspath(DIRECTORY, JOBLIB_FILE)
lgb_model = joblib.load(joblib_path)
lgb_model

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.05,
               metric='binary_logloss', objective='binary', subgroup=0.8,
               subsample=0.8, verbose=-1)

## 二、运行推理 

In [6]:
# 假设这行数据长这样
csv_file = util.gen_abspath(DIRECTORY, TEST_FILE)
df_test = util.read_csv(csv_file, sep=',', header=None)
df_test.columns=COLS

df_one = df_test.iloc[3:4,:]

# 转成特征与标号
XX = df_one.drop(LABEL_COL, axis=1)  # features
yy = df_one[LABEL_COL].apply(lambda e: 0 if e == ' <=50K.' else 1)  # label

# 处理类别特征
kkv_path = util.gen_abspath(DIRECTORY, KKV_FILE)
XX, _ = util.load_label_encoder(XX, kkv_path)
XX

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
3,44,4,160323,15,10,2,7,0,2,1,7688,0,40,39


In [7]:
lgb_model.predict(XX)[0]

1

In [8]:
# 评估模型性能
score = lgb_model.score(XX, yy)
score

1.0

## 三、存成 PMML

将 `LGBMClassifier` 存成 PMML 格式

In [9]:
pipeline = Pipeline([
    ('lgbm', lgb_model)
])

In [10]:
col_names = COLS
if LABEL_COL in col_names:
    col_names.remove(LABEL_COL)
target_name = LABEL_COL

In [11]:
# Convert to PMML
pmml_path = util.gen_abspath(DIRECTORY, PMML_FILE)
lgb_to_pmml(pipeline, col_names=col_names, target_name=target_name, pmml_f_name=pmml_path)